# Vector space with ML

This lab will be devoted to the use of ML model for the needs of information retrieval and text classification.  

**Searching in the curious facts database**

The facts dataset is given [here](https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt), take a look. We want you to retrieve facts **relevant to the query** (whatever it means), for example, you type "good mood", and get to know that Cherophobia is the fear of fun. For this, the idea is to utilize document vectors. However, instead of forming vectors with tf-idf and reducing dimensions, this time we want to obtain fixed-size vectors for documents using ML model.

## 1. Use neural networks to embed sentences

Make use of any, starting from doc2vec up to Transformers, etc. Provide all code, dependencies, installation requirements.


- [UCE in spacy 2](https://spacy.io/universe/project/spacy-universal-sentence-encoder) (`!pip install spacy-universal-sentence-encoder`)
- [Sentence BERT in spacy 2](https://spacy.io/universe/project/spacy-sentence-bert) (`!pip install spacy-sentence-bert`)
- [Pretrained 🤗 Transformers](https://huggingface.co/transformers/pretrained_models.html)
- [Spacy 3 transformers](https://spacy.io/usage/embeddings-transformers#transformers-installation)
- [doc2vec pretrained](https://github.com/jhlau/doc2vec)
- [Some more sentence transformers](https://www.sbert.net/docs/quickstart.html)
- [Even fasttext can do a sentence embedding](https://fasttext.cc/docs/en/python-module.html#model-object)

Here should be dependency installation, download instructions and so on. With outputs.

In [ ]:
!pip install spacy-sentence-bert
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/en_stsb_roberta_base-0.1.2.tar.gz#en_stsb_roberta_base-0.1.2
### Restart the runtime after installing there packages

And then use the library to download (and load) the model:

NB: model downloading may take time (depending on the model hosting). If you think it may take a long time, ask your TA for assistance with binaries.

In [ ]:

import spacy_sentence_bert
# load one of the models listed at https://github.com/MartinoMensio/spacy-sentence-bert/

# nlp = spacy_sentence_bert.load_model('en_roberta_large_nli_stsb_mean_tokens')
nlp = spacy_sentence_bert.load_model('en_stsb_roberta_large')
# get two documents
doc_1 = nlp('Hi there, how are you?')
doc_2 = nlp('Hello there, how are you doing today?')
# use the similarity method that is based on the vectors, on Doc, Span or Token
print(doc_1.similarity(doc_2[0:7]))

0.7491615722715862


## 2. Write a function that prepares embedding of arbitrary queries

Write a function, which returns a fixed-sized vector of embedding.

In [ ]:
from scipy.spatial.distance import cosine
cosine(doc_1.vector, doc_2.vector)

0.3481746315956116

In [ ]:
def embed(text):
    # return ft.get_sentence_vector(text)
    return nlp(text).vector

Here we check that embeddings are of the same size and type.

In [ ]:
assert embed(
            "Some random text"
        ).shape == \
        embed(
            "Folks, here's a story about Minnie the Moocher. "
            "She was a lowdown hoochie coocher. "
            "She was the roughest, toughest frail, "
            "but Minnie had a heart as big as a whale"
        ).shape, "Shape should match"

NB: here we check DISTANCE, not similarity. This similar texts should produce results close to 0.

In [ ]:
from scipy.spatial.distance import cosine

assert abs(cosine(
            embed("some text for testing"),
            embed("some text for testing")
        )) < 1e-4, "Embedding should match"

assert abs(cosine(
            embed("Cats eat mice."),
            embed("Terminator is an autonomous cyborg, typically humanoid, originally conceived as a virtually indestructible soldier, infiltrator, and assassin.")
        )) > 0.2, "Embeddings should be far"

## 3. Read the data

Now, let's read the facts dataset. Download it from the abovementioned url and read to the list of sentences.

In [ ]:
import requests
url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"

#TODO read facts into a list of facts. Each fact is a separate element of array
facts = []

facts = requests.get(url).text.split('\n')

In [ ]:
print(*facts[:5], sep='\n')

assert len(facts) == 159
assert ('our lovely little planet') in facts[0]

1. If you somehow found a way to extract all of the gold from the bubbling core of our lovely little planet, you would be able to cover all of the land in a layer of gold up to your knees.
2. McDonalds calls frequent buyers of their food "heavy users."
3. The average person spends 6 months of their lifetime waiting on a red light to turn green.
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
5. You burn more calories sleeping than you do watching television.


## 4. Transform sentences to vectors

Transform the list of facts to `numpy.array` of vectors corresponding to each document (`sent_vecs`), inferring them from the model we just loaded.

In [ ]:
import numpy as np
#TODO infer vectors

sent_vecs = np.array([])
sent_vecs = np.array([embed(f) for f in facts])

In [ ]:
assert sent_vecs.shape[0] == len(facts)

## 5. Find closest to the query

Now find 5 facts which are the closest to the query using cosine measure.

### 5.1. Closest search

In [ ]:
def find_k_closest(query, dataset, k=10):
    # TODO your code gere
    return np.argsort(dataset @ query)[-k:]
    return range(k)

### 5.1. Use your function

In [ ]:
query = "good mood"
query_vec = embed(query)

print("Results for query:", query)
print()
for k in find_k_closest(query_vec, sent_vecs, 5):
    print("\t", facts[k])

Results for query: good mood

	 29. Chewing gum while you cut an onion will help keep you from crying.
	 1. If you somehow found a way to extract all of the gold from the bubbling core of our lovely little planet, you would be able to cover all of the land in a layer of gold up to your knees.
	 70. Pirates wore earrings because they believed it improved their eyesight.
	 64. In Japan, crooked teeth are considered cute and attractive.
	 57. Gorillas burp when they are happy


## 6. Measure DCG@5 for the following query bucket
```
good mood
gorilla
woman
earth
japan
people
math
```

Recommend 5 facts to each of the queries. Write your code below.

In [ ]:
bucket = """good mood
gorilla
woman
earth
japan
people
math""".split('\n')

for term in bucket:
    print(term)
    for k in find_k_closest(embed(term), sent_vecs, k=5)[::-1]:
        print("\t", facts[k])

good mood
	 60. It is considered good luck in Japan when a sumo wrestler makes your baby cry.
	 10. If you believe that you're truly one in a million, there are still approximately 7,184 more people out there just like you.
	 116. Male dogs lift their legs when they are urinating for a reason. They are trying to leave their mark higher so that it gives off the message that they are tall and intimidating.
	 57. Gorillas burp when they are happy
	 84. You are 1% shorter in the evening than in the morning
gorilla
	 106. The male ostrich can roar just like a lion.
	 85. The elephant is the only mammal that can't jump!
	 107. Mountain lions can whistle.
	 57. Gorillas burp when they are happy
	 139. Beetles taste like apples, wasps like pine nuts, and worms like fried bacon.
woman
	 131. If a pregnant woman has organ damage, the baby in her womb sends stem cells to help repair the organ.
	 65. A Swedish woman lost her wedding ring, and found it 16 years later- growing on a carrot in her gar